# PDF Extraction Test Script

Script này sẽ:
1. Đọc thông tin CV từ Resume.json
2. Duyệt từng file PDF trong folder resumes
3. Gọi API pdfController để trích xuất dữ liệu
4. So sánh kết quả với dữ liệu trong Resume.json


In [1]:
# Import libraries
import json
import os
import requests
from pathlib import Path
from typing import Dict, List, Any
import pandas as pd
from datetime import datetime


In [3]:
# Configuration
BASE_DIR = Path.cwd()
RESUME_JSON_PATH = BASE_DIR / "Resume.json"
RESUMES_FOLDER = BASE_DIR / "resumes"

# API Configuration
API_BASE_URL = "http://localhost:3005"
API_EXTRACT_ENDPOINT = f"{API_BASE_URL}/api/v1/pdf/extract"

print(f"Base directory: {BASE_DIR}")
print(f"Resume JSON path: {RESUME_JSON_PATH}")
print(f"Resumes folder: {RESUMES_FOLDER}")
print(f"API endpoint: {API_EXTRACT_ENDPOINT}")


Base directory: /Users/gothartnguyen/Documents/Gothart/DAI_HOC_2021_2026/TOT_NGHIEP/Jobhunter_AiServer/Jobhunter_AiServer
Resume JSON path: /Users/gothartnguyen/Documents/Gothart/DAI_HOC_2021_2026/TOT_NGHIEP/Jobhunter_AiServer/Jobhunter_AiServer/Resume.json
Resumes folder: /Users/gothartnguyen/Documents/Gothart/DAI_HOC_2021_2026/TOT_NGHIEP/Jobhunter_AiServer/Jobhunter_AiServer/resumes
API endpoint: http://localhost:3005/api/v1/pdf/extract


In [4]:
# Đọc Resume.json
def load_resume_data(json_path: Path) -> List[Dict[str, Any]]:
    """Đọc và parse file Resume.json"""
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✓ Đã đọc {len(data)} CV từ Resume.json")
        return data
    except FileNotFoundError:
        print(f"✗ Không tìm thấy file: {json_path}")
        return []
    except json.JSONDecodeError as e:
        print(f"✗ Lỗi parse JSON: {e}")
        return []

resume_data = load_resume_data(RESUME_JSON_PATH)

# Hiển thị thông tin các CV
if resume_data:
    print("\nDanh sách CV trong Resume.json:")
    for idx, cv in enumerate(resume_data, 1):
        print(f"{idx}. {cv.get('full_name', 'N/A')} - {cv.get('email', 'N/A')}")


✓ Đã đọc 3 CV từ Resume.json

Danh sách CV trong Resume.json:
1. Pham Duc Thanh - duybaoandinh@gmail.com
2. Nguyen Minh Triet - duybaoandinh@gmail.com
3. Vo Thi My Linh - duybaoandinh@gmail.com


In [5]:
# Mapping file names với CV data
# Dựa trên tên file và thông tin trong Resume.json
FILE_TO_CV_MAPPING = {
    "Data_Engineer.pdf": "Pham Duc Thanh",
    "FullStack_Web_Developer.pdf": "Nguyen Minh Triet",
    "Front_End_Developer.pdf": "Vo Thi My Linh"
}

def find_cv_by_name(name: str, resume_list: List[Dict]) -> Dict[str, Any]:
    """Tìm CV trong danh sách dựa trên tên"""
    for cv in resume_list:
        if cv.get('full_name') == name:
            return cv
    return None

print("Mapping file names:")
for filename, name in FILE_TO_CV_MAPPING.items():
    cv = find_cv_by_name(name, resume_data)
    if cv:
        print(f"  {filename} -> {name} ({cv.get('email', 'N/A')})")
    else:
        print(f"  {filename} -> {name} (NOT FOUND)")


Mapping file names:
  Data_Engineer.pdf -> Pham Duc Thanh (duybaoandinh@gmail.com)
  FullStack_Web_Developer.pdf -> Nguyen Minh Triet (duybaoandinh@gmail.com)
  Front_End_Developer.pdf -> Vo Thi My Linh (duybaoandinh@gmail.com)


In [6]:
# Kiểm tra folder resumes
def get_pdf_files(folder_path: Path) -> List[Path]:
    """Lấy danh sách các file PDF trong folder"""
    if not folder_path.exists():
        print(f"✗ Folder không tồn tại: {folder_path}")
        return []
    
    pdf_files = list(folder_path.glob("*.pdf"))
    print(f"✓ Tìm thấy {len(pdf_files)} file PDF trong folder resumes")
    for pdf_file in pdf_files:
        print(f"  - {pdf_file.name} ({pdf_file.stat().st_size / 1024:.2f} KB)")
    return pdf_files

pdf_files = get_pdf_files(RESUMES_FOLDER)


✓ Tìm thấy 3 file PDF trong folder resumes
  - Front_End_Developer.pdf (34.03 KB)
  - Data_Engineer.pdf (33.81 KB)
  - FullStack_Web_Developer.pdf (35.34 KB)


In [7]:
# Hàm gọi API extract PDF
def extract_pdf_via_api(file_path: Path, api_url: str) -> Dict[str, Any]:
    """Gọi API để trích xuất dữ liệu từ PDF"""
    try:
        print(f"\n📄 Đang xử lý: {file_path.name}")
        
        # Mở file và gửi request
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.name, f, 'application/pdf')}
            data = {'pageNumber': 0}
            
            response = requests.post(api_url, files=files, data=data, timeout=120)
            
            if response.status_code == 200:
                result = response.json()
                if result.get('success'):
                    print(f"  ✓ Trích xuất thành công")
                    return result.get('data', {}).get('extractedData', {})
                else:
                    print(f"  ✗ API trả về success=false: {result.get('message', 'Unknown error')}")
                    return None
            elif response.status_code == 403:
                print(f"  ✗ PDF Extractor đang bị vô hiệu hóa")
                return None
            else:
                print(f"  ✗ Lỗi HTTP {response.status_code}: {response.text}")
                return None
                
    except requests.exceptions.Timeout:
        print(f"  ✗ Timeout khi gọi API")
        return None
    except requests.exceptions.ConnectionError:
        print(f"  ✗ Không thể kết nối đến API. Đảm bảo server đang chạy tại {api_url}")
        return None
    except Exception as e:
        print(f"  ✗ Lỗi: {str(e)}")
        return None

# Test kết nối API
print("\n🔍 Kiểm tra kết nối API...")
try:
    health_check = requests.get(f"{API_BASE_URL}/api/v1/health", timeout=5)
    if health_check.status_code == 200:
        print(f"✓ API server đang hoạt động")
    else:
        print(f"⚠ API server trả về status {health_check.status_code}")
except Exception as e:
    print(f"✗ Không thể kết nối đến API: {e}")
    print(f"  Đảm bảo server đang chạy tại {API_BASE_URL}")



🔍 Kiểm tra kết nối API...
✓ API server đang hoạt động


In [8]:
# Duyệt từng file PDF và gọi API
results = []

for pdf_file in pdf_files:
    # Tìm CV tương ứng trong Resume.json
    cv_name = FILE_TO_CV_MAPPING.get(pdf_file.name)
    expected_cv = find_cv_by_name(cv_name, resume_data) if cv_name else None
    
    # Gọi API extract
    extracted_data = extract_pdf_via_api(pdf_file, API_EXTRACT_ENDPOINT)
    
    # Lưu kết quả
    result_entry = {
        'filename': pdf_file.name,
        'expected_cv_name': cv_name,
        'extracted_data': extracted_data,
        'expected_data': expected_cv,
        'timestamp': datetime.now().isoformat()
    }
    results.append(result_entry)

print(f"\n✅ Đã xử lý {len(results)} file PDF")



📄 Đang xử lý: Front_End_Developer.pdf
  ✓ Trích xuất thành công

📄 Đang xử lý: Data_Engineer.pdf
  ✓ Trích xuất thành công

📄 Đang xử lý: FullStack_Web_Developer.pdf
  ✓ Trích xuất thành công

✅ Đã xử lý 3 file PDF


In [9]:
# So sánh kết quả trích xuất với dữ liệu trong Resume.json

def normalize_text(value):
    if isinstance(value, str):
        return value.strip().lower()
    return value

def compare_fields(extracted: Dict, expected: Dict, field: str) -> Dict[str, Any]:
    """So sánh một field giữa extracted và expected"""
    extracted_value = normalize_text(extracted.get(field))
    expected_value = normalize_text(expected.get(field))
    
    match = extracted_value == expected_value
    
    return {
        'field': field,
        'extracted': extracted.get(field),
        'expected': expected.get(field),
        'match': match
    }

def compare_projects(extracted_projects: List[Dict[str, Any]], expected_projects: List[Dict[str, Any]]) -> Dict[str, Any]:
    """So sánh danh sách projects"""
    project_results = []
    matched_names = 0
    used_indices = set()
    
    for expected_proj in expected_projects:
        expected_name_norm = normalize_text(expected_proj.get('project_name'))
        match = None
        match_index = None
        
        for idx, extracted_proj in enumerate(extracted_projects):
            if idx in used_indices:
                continue
            if normalize_text(extracted_proj.get('project_name')) == expected_name_norm:
                match = extracted_proj
                match_index = idx
                break
        
        expected_languages = set(normalize_text(lang) for lang in (expected_proj.get('languages') or []) if lang)
        expected_responsibilities = set(normalize_text(resp) for resp in (expected_proj.get('responsibilities') or []) if resp)
        
        project_comp = {
            'expected_project_name': expected_proj.get('project_name'),
            'match_found': match is not None,
            'extracted_project_name': match.get('project_name') if match else None,
            'role_match': False,
            'duration_match': False,
            'team_size_match': False,
            'description_match': False,
            'languages_expected_count': len(expected_languages),
            'languages_extracted_count': 0,
            'languages_matched': 0,
            'languages_overlap': 0,
            'responsibilities_expected_count': len(expected_responsibilities),
            'responsibilities_extracted_count': 0,
            'responsibilities_matched': 0,
            'responsibilities_overlap': 0
        }
        
        if match:
            used_indices.add(match_index)
            matched_names += 1
            extracted_languages = set(normalize_text(lang) for lang in (match.get('languages') or []) if lang)
            extracted_responsibilities = set(normalize_text(resp) for resp in (match.get('responsibilities') or []) if resp)
            
            project_comp.update({
                'role_match': normalize_text(match.get('role')) == normalize_text(expected_proj.get('role')),
                'duration_match': normalize_text(match.get('duration')) == normalize_text(expected_proj.get('duration')),
                'team_size_match': match.get('team_size') == expected_proj.get('team_size'),
                'description_match': normalize_text(match.get('description')) == normalize_text(expected_proj.get('description')),
                'languages_extracted_count': len(extracted_languages),
                'languages_matched': len(expected_languages & extracted_languages),
                'languages_overlap': len(expected_languages & extracted_languages) / max(len(expected_languages), 1) if expected_languages else 0,
                'responsibilities_extracted_count': len(extracted_responsibilities),
                'responsibilities_matched': len(expected_responsibilities & extracted_responsibilities),
                'responsibilities_overlap': len(expected_responsibilities & extracted_responsibilities) / max(len(expected_responsibilities), 1) if expected_responsibilities else 0
            })
        
        project_results.append(project_comp)
    
    project_name_match_rate = matched_names / max(len(expected_projects), 1) if expected_projects else 0
    
    return {
        'project_comparisons': project_results,
        'projects_name_match_count': matched_names,
        'total_expected_projects': len(expected_projects),
        'project_name_match_rate': project_name_match_rate,
        'extracted_projects_extra': max(len(extracted_projects) - len(expected_projects), 0)
    }

def compare_cv_data(extracted: Dict, expected: Dict) -> Dict[str, Any]:
    """So sánh toàn bộ dữ liệu CV"""
    if not extracted or not expected:
        return {'error': 'Missing data for comparison'}
    
    # Các field cần so sánh
    fields_to_compare = [
        'full_name',
        'email',
        'phone_number',
        'objective',
        'university',
        'certificate'
    ]
    
    comparisons = {}
    matches = 0
    total = len(fields_to_compare)
    
    for field in fields_to_compare:
        comp = compare_fields(extracted, expected, field)
        comparisons[field] = comp
        if comp['match']:
            matches += 1
    
    # So sánh technical_skills (list)
    extracted_skills = set(normalize_text(s) for s in (extracted.get('technical_skills') or []) if s)
    expected_skills = set(normalize_text(s) for s in (expected.get('technical_skills') or []) if s)
    skills_match = len(extracted_skills & expected_skills) / max(len(expected_skills), 1) if expected_skills else 0
    
    # So sánh projects chi tiết
    extracted_projects_list = extracted.get('projects') or []
    expected_projects_list = expected.get('projects') or []
    project_comparison = compare_projects(extracted_projects_list, expected_projects_list)
    
    # So sánh số lượng projects
    extracted_projects = len(extracted_projects_list)
    expected_projects = len(expected_projects_list)
    projects_count_match = extracted_projects == expected_projects
    
    return {
        'field_comparisons': comparisons,
        'matches': matches,
        'total_fields': total,
        'match_rate': matches / total if total > 0 else 0,
        'skills_overlap': skills_match,
        'projects_count_match': projects_count_match,
        'extracted_projects_count': extracted_projects,
        'expected_projects_count': expected_projects,
        **project_comparison
    }

# Thực hiện so sánh cho từng file
comparison_results = []

for result in results:
    if result['extracted_data'] and result['expected_data']:
        comparison = compare_cv_data(result['extracted_data'], result['expected_data'])
        comparison['filename'] = result['filename']
        comparison['cv_name'] = result['expected_cv_name']
        comparison_results.append(comparison)
        
        print(f"\n📊 So sánh: {result['filename']}")
        print(f"  CV: {result['expected_cv_name']}")
        print(f"  Match rate: {comparison['match_rate']*100:.1f}% ({comparison['matches']}/{comparison['total_fields']})")
        print(f"  Skills overlap: {comparison['skills_overlap']*100:.1f}%")
        print(f"  Projects count: {comparison['extracted_projects_count']} (expected: {comparison['expected_projects_count']}) {'✓' if comparison['projects_count_match'] else '✗'}")
        print(f"  Project name matches: {comparison['projects_name_match_count']}/{comparison['total_expected_projects']} ({comparison['project_name_match_rate']*100:.1f}%)")
        if comparison['extracted_projects_extra']:
            print(f"  Extra extracted projects: {comparison['extracted_projects_extra']}")
    else:
        print(f"\n⚠ Không thể so sánh {result['filename']} (thiếu dữ liệu)")



📊 So sánh: Front_End_Developer.pdf
  CV: Vo Thi My Linh
  Match rate: 83.3% (5/6)
  Skills overlap: 100.0%
  Projects count: 3 (expected: 3) ✓
  Project name matches: 2/3 (66.7%)

📊 So sánh: Data_Engineer.pdf
  CV: Pham Duc Thanh
  Match rate: 100.0% (6/6)
  Skills overlap: 100.0%
  Projects count: 3 (expected: 3) ✓
  Project name matches: 3/3 (100.0%)

📊 So sánh: FullStack_Web_Developer.pdf
  CV: Nguyen Minh Triet
  Match rate: 83.3% (5/6)
  Skills overlap: 100.0%
  Projects count: 3 (expected: 3) ✓
  Project name matches: 3/3 (100.0%)


In [10]:
# Hiển thị chi tiết so sánh cho từng CV (bao gồm projects)
if comparison_results:
    for comp_result in comparison_results:
        print(f"\n{'='*60}")
        print(f"File: {comp_result['filename']}")
        print(f"CV: {comp_result['cv_name']}")
        print(f"{'='*60}")
        
        for field, comp in comp_result['field_comparisons'].items():
            status = "✓" if comp['match'] else "✗"
            print(f"\n{status} {field}:")
            print(f"  Extracted: {comp['extracted']}")
            print(f"  Expected:  {comp['expected']}")
        
        if comp_result.get('project_comparisons'):
            print("\n--- Project comparisons ---")
            for idx, proj_comp in enumerate(comp_result['project_comparisons'], 1):
                match_icon = "✓" if proj_comp['match_found'] else "✗"
                print(f"\nProject {idx}: {proj_comp['expected_project_name']}")
                print(f"  Match found: {match_icon}")
                if proj_comp['match_found']:
                    print(f"  Extracted project name: {proj_comp['extracted_project_name']}")
                    print(f"  Role match: {'✓' if proj_comp['role_match'] else '✗'}")
                    print(f"  Duration match: {'✓' if proj_comp['duration_match'] else '✗'}")
                    print(f"  Team size match: {'✓' if proj_comp['team_size_match'] else '✗'}")
                    print(f"  Description match: {'✓' if proj_comp['description_match'] else '✗'}")
                    print(f"  Languages matched: {proj_comp['languages_matched']}/{proj_comp['languages_expected_count']} ({proj_comp['languages_overlap']*100:.1f}%)")
                    print(f"  Responsibilities matched: {proj_comp['responsibilities_matched']}/{proj_comp['responsibilities_expected_count']} ({proj_comp['responsibilities_overlap']*100:.1f}%)")
                else:
                    print("  -> Không tìm thấy project tương ứng trong dữ liệu trích xuất")
else:
    print("⚠️ Không có dữ liệu trong comparison_results")



File: Front_End_Developer.pdf
CV: Vo Thi My Linh

✓ full_name:
  Extracted: Vo Thi My Linh
  Expected:  Vo Thi My Linh

✗ email:
  Extracted: duybaoanich@gmail.com
  Expected:  duybaoandinh@gmail.com

✓ phone_number:
  Extracted: 0345678123
  Expected:  0345678123

✓ objective:
  Extracted: Front-End Developer specializing in building responsive, accessible, and high-performance web interfaces. Experienced in implementing scalable component architectures and modern JavaScript frameworks.
  Expected:  Front-End Developer specializing in building responsive, accessible, and high-performance web interfaces. Experienced in implementing scalable component architectures and modern JavaScript frameworks.

✓ university:
  Extracted: University of Technology and Education - The University of Da Nang
  Expected:  University of Technology and Education - The University of Da Nang

✓ certificate:
  Extracted: Meta Front-End Developer Professional Certificate
  Expected:  Meta Front-End Developer 

In [11]:
# Tạo bảng tổng hợp kết quả
if comparison_results:
    summary_data = []
    for comp in comparison_results:
        summary_data.append({
            'File': comp['filename'],
            'CV Name': comp['cv_name'],
            'Match Rate': f"{comp['match_rate']*100:.1f}%",
            'Skills Overlap': f"{comp['skills_overlap']*100:.1f}%",
            'Projects Match': '✓' if comp['projects_count_match'] else '✗',
            'Project Name Match Rate': f"{comp['project_name_match_rate']*100:.1f}%",
            'Extracted Projects': comp['extracted_projects_count'],
            'Expected Projects': comp['expected_projects_count']
        })
    
    df = pd.DataFrame(summary_data)
    print("\n📋 Tổng hợp kết quả:")
    print(df.to_string(index=False))



📋 Tổng hợp kết quả:
                       File           CV Name Match Rate Skills Overlap Projects Match Project Name Match Rate  Extracted Projects  Expected Projects
    Front_End_Developer.pdf    Vo Thi My Linh      83.3%         100.0%              ✓                   66.7%                   3                  3
          Data_Engineer.pdf    Pham Duc Thanh     100.0%         100.0%              ✓                  100.0%                   3                  3
FullStack_Web_Developer.pdf Nguyen Minh Triet      83.3%         100.0%              ✓                  100.0%                   3                  3


In [ ]:
# Lưu kết quả ra file JSON
output_file = BASE_DIR / f"extraction_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

output_data = {
    'timestamp': datetime.now().isoformat(),
    'api_endpoint': API_EXTRACT_ENDPOINT,
    'total_files': len(results),
    'results': results,
    'comparisons': comparison_results
}

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n💾 Đã lưu kết quả vào: {output_file}")


In [12]:
# So sánh chi tiết Responsibilities cho từng project theo định dạng từng dòng
# Dựa trên dữ liệu "results" đã có (extracted_data và expected_data)

def find_project_by_name(projects, name):
    name_norm = normalize_text(name)
    for p in projects or []:
        if normalize_text(p.get('project_name')) == name_norm:
            return p
    return None

for result in results:
    extracted = result.get('extracted_data') or {}
    expected = result.get('expected_data') or {}
    
    expected_projects = expected.get('projects') or []
    extracted_projects = extracted.get('projects') or []
    
    print(f"\n{'='*60}")
    print(f"File: {result.get('filename')}")
    print(f"CV: {result.get('expected_cv_name')}")
    print(f"{'='*60}")

    for idx, exp_proj in enumerate(expected_projects, 1):
        print(f"\n--- Project {idx}: {exp_proj.get('project_name')} ---")
        matched_proj = find_project_by_name(extracted_projects, exp_proj.get('project_name'))
        
        exp_resps = exp_proj.get('responsibilities') or []
        ext_resps = matched_proj.get('responsibilities') if matched_proj else []
        
        ext_norm_to_raw = {normalize_text(r): r for r in (ext_resps or [])}
        ext_norm_set = set(ext_norm_to_raw.keys())
        
        if not exp_resps:
            print("(Không có responsibilities trong dữ liệu kỳ vọng)")
        
        print("\nResponsibilities:")
        for r in exp_resps:
            rn = normalize_text(r)
            matched = rn in ext_norm_set
            icon = '✓' if matched else '✗'
            extracted_line = ext_norm_to_raw.get(rn)
            print(f"\n{icon} responsibility:")
            print(f"  Extracted: {extracted_line if extracted_line is not None else '—'}")
            print(f"  Expected:  {r}")
        
        # Liệt kê responsibilities thừa (có trong extracted nhưng không có trong expected)
        extra = [raw for norm, raw in ext_norm_to_raw.items() if norm not in set(normalize_text(x) for x in exp_resps)]
        if extra:
            print("\n(Responsibilities thêm trong dữ liệu trích xuất không có trong kỳ vọng):")
            for er in extra:
                print(f"  + {er}")



File: Front_End_Developer.pdf
CV: Vo Thi My Linh

--- Project 1: NovaUI - Design System and Component Library ---

Responsibilities:

✗ responsibility:
  Extracted: —
  Expected:  The system provided a unified UI component framework compatible with React applications using TypeScript for type safety.

✗ responsibility:
  Extracted: —
  Expected:  Storybook was used for component documentation, interactive demos, and visual testing integration.

✗ responsibility:
  Extracted: —
  Expected:  Design tokens were introduced to synchronize colors, typography, and spacing across projects.

✗ responsibility:
  Extracted: —
  Expected:  Sass mixins and utility classes were implemented for customizable theming and responsiveness.

✗ responsibility:
  Extracted: —
  Expected:  Automated visual regression testing was configured using Chromatic CI integration.

✗ responsibility:
  Extracted: —
  Expected:  The library supported accessibility standards including ARIA roles and keyboard navigation c

In [14]:
# Tổng hợp độ chính xác (matching %) của PDF Extractor
# Tính điểm tổng hợp cho từng CV và điểm trung bình toàn bộ 
from statistics import mean

summary_rows = []

if comparison_results:
    for comp in comparison_results:
        fields_score = comp.get('match_rate', 0.0)  # tỉ lệ match các field cơ bản
        skills_score = comp.get('skills_overlap', 0.0)  # tỉ lệ overlap kỹ năng
        proj_name_score = comp.get('project_name_match_rate', 0.0)  # tỉ lệ khớp tên project
        proj_count_score = 1.0 if comp.get('projects_count_match') else 0.0  # khớp số lượng project
        
        # Trung bình languages_overlap trên tất cả project (nếu có)
        languages_overlaps = []
        for p in comp.get('project_comparisons', []) or []:
            if isinstance(p.get('languages_overlap'), (int, float)):
                languages_overlaps.append(p['languages_overlap'])
        lang_avg = mean(languages_overlaps) if languages_overlaps else 0.0
        
        # Công thức tổng hợp (loại bỏ Responsibilities)
        components = [fields_score, skills_score, proj_name_score, proj_count_score, lang_avg]
        overall_accuracy = mean(components)
        
        summary_rows.append({
            'File': comp.get('filename'),
            'CV Name': comp.get('cv_name'),
            'Fields Match': f"{fields_score*100:.1f}%",
            'Skills Overlap': f"{skills_score*100:.1f}%",
            'Project Name Match': f"{proj_name_score*100:.1f}%",
            'Projects Count Match': '✓' if proj_count_score == 1.0 else '✗',
            'Languages Avg Overlap': f"{lang_avg*100:.1f}%",
            'Overall Accuracy': overall_accuracy  # giữ dạng số để tính trung bình, hiển thị sau
        })
    
    # In bảng kết quả
    df_summary = pd.DataFrame(summary_rows)
    # Hiển thị Overall Accuracy dạng %
    if not df_summary.empty:
        df_print = df_summary.copy()
        df_print['Overall Accuracy'] = (df_print['Overall Accuracy'] * 100).map(lambda x: f"{x:.1f}%")
        print("\n🎯 Kết quả độ chính xác theo từng CV:")
        print(df_print.to_string(index=False))
        
        # Tính trung bình toàn bộ
        overall_mean = float(mean(df_summary['Overall Accuracy'])) if not df_summary.empty else 0.0
        print(f"\n✅ Độ chính xác trung bình (Overall Accuracy Mean): {overall_mean*100:.1f}%")
else:
    print("⚠️ Chưa có comparison_results. Hãy chạy các cell phía trên trước.")



🎯 Kết quả độ chính xác theo từng CV:
                       File           CV Name Fields Match Skills Overlap Project Name Match Projects Count Match Languages Avg Overlap Overall Accuracy
    Front_End_Developer.pdf    Vo Thi My Linh        83.3%         100.0%              66.7%                    ✓                 66.7%            83.3%
          Data_Engineer.pdf    Pham Duc Thanh       100.0%         100.0%             100.0%                    ✓                100.0%           100.0%
FullStack_Web_Developer.pdf Nguyen Minh Triet        83.3%         100.0%             100.0%                    ✓                 83.3%            93.3%

✅ Độ chính xác trung bình (Overall Accuracy Mean): 92.2%
